In [42]:
import tensorflow as tf
import numpy as np

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 80.2M    0 32768    0     0  19363      0  1:12:24  0:00:01  1:12:23 19366
  0 80.2M    0  208k    0     0  80222      0  0:17:28  0:00:02  0:17:26 80222
  0 80.2M    0  448k    0     0   121k      0  0:11:13  0:00:03  0:11:10  121k
  0 80.2M    0  752k    0     0   160k      0  0:08:30  0:00:04  0:08:26  160k
  1 80.2M    1 1280k    0     0   226k      0  0:06:02  0:00:05  0:05:57  267k
  2 80.2M    2 1968k    0     0   296k      0  0:04:37  0:00:06  0:04:31  390k
  3 80.2M    3 2992k    0     0   391k      0  0:03:29  0:00:07  0:03:22  557k
  5 80.2M    5 4224k    0     0   488k      0  0:02:48  0:00:08  0:02:40  758k
  6 80.2M    6 5536k    0     0   574k      0  0:02

In [3]:
%ls aclImdb

 Le volume dans le lecteur C n'a pas de nom.
 Le num‚ro de s‚rie du volume est 8011-DD18

 R‚pertoire de C:\Users\Setup Game\Desktop\Chat-Voice bots\NeuralNet_venv\Scripts\aclImdb

21/11/2022  18:42    <DIR>          .
21/11/2022  18:42    <DIR>          ..
12/04/2011  18:14           845ÿ980 imdb.vocab
11/06/2011  23:54           903ÿ029 imdbEr.txt
26/06/2011  01:18             4ÿ037 README
21/11/2022  18:42    <DIR>          test
21/11/2022  18:42    <DIR>          train
               3 fichier(s)        1ÿ753ÿ046 octets
               4 R‚p(s)  42ÿ023ÿ432ÿ192 octets libres


In [4]:
%ls aclImdb\test

 Le volume dans le lecteur C n'a pas de nom.
 Le num‚ro de s‚rie du volume est 8011-DD18

 R‚pertoire de C:\Users\Setup Game\Desktop\Chat-Voice bots\NeuralNet_venv\Scripts\aclImdb\test

21/11/2022  18:42    <DIR>          .
21/11/2022  18:42    <DIR>          ..
12/04/2011  18:25        20ÿ205ÿ283 labeledBow.feat
21/11/2022  18:42    <DIR>          neg
21/11/2022  18:42    <DIR>          pos
12/04/2011  10:48           612ÿ500 urls_neg.txt
12/04/2011  10:48           612ÿ500 urls_pos.txt
               3 fichier(s)       21ÿ430ÿ283 octets
               4 R‚p(s)  42ÿ023ÿ432ÿ192 octets libres


In [5]:
%ls aclImdb\train

 Le volume dans le lecteur C n'a pas de nom.
 Le num‚ro de s‚rie du volume est 8011-DD18

 R‚pertoire de C:\Users\Setup Game\Desktop\Chat-Voice bots\NeuralNet_venv\Scripts\aclImdb\train

21/11/2022  18:42    <DIR>          .
21/11/2022  18:42    <DIR>          ..
12/04/2011  18:17        21ÿ021ÿ197 labeledBow.feat
21/11/2022  18:42    <DIR>          neg
21/11/2022  18:42    <DIR>          pos
21/11/2022  18:43    <DIR>          unsup
12/04/2011  18:22        41ÿ348ÿ699 unsupBow.feat
12/04/2011  10:48           612ÿ500 urls_neg.txt
12/04/2011  10:48           612ÿ500 urls_pos.txt
12/04/2011  10:47         2ÿ450ÿ000 urls_unsup.txt
               5 fichier(s)       66ÿ044ÿ896 octets
               5 R‚p(s)  42ÿ023ÿ432ÿ192 octets libres


<h5>Cet exemple montre comment faire de la classification de texte à partir de texte brut (comme un ensemble de fichiers texte sur le disque). Nous démontrons le flux de travail sur le jeu de données de classification de sentiments IMDB. Nous utilisons la couche TextVectorization pour le découpage des mots et l'indexation.<h5>

In [54]:
!type aclImdb\train\pos\6248_7.txt


Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

In [35]:
import shutil

shutil.rmtree('./aclImdb/train/unsup', ignore_errors=True)

<h5>nous pouvons utiliser l'utilitaire tf.keras.preprocessing.text_dataset_from_directory pour générer un objet tf.data.Dataset étiqueté à partir d'un ensemble de fichiers texte sur le disque classés dans des dossiers spécifiques à la classe.

Nous allons l'utiliser pour générer les ensembles de données de formation, de validation et de test. Les ensembles de données de validation et d'apprentissage sont générés à partir de deux sous-ensembles du répertoire train, avec 20% des échantillons allant à l'ensemble de données de validation et 80% allant à l'ensemble de données d'apprentissage.


Lors de l'utilisation des arguments validation_split & subset, nous devons nous assurer de spécifier une graine aléatoire, ou de passer shuffle=False, afin que les splits de validation et de formation que vous obtenez ne se chevauchent pas.<h5>

In [46]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


<h5>Voyons quelques exemples :<h5>

In [47]:
# It's important to take a look at your raw data to ensure your normalization
# and tokenization will work as expected. We can do that by taking a few
# examples from the training set and looking at them.
# This is one of the places where eager execution shines:
# we can just evaluate these tensors using .numpy()
# instead of needing to evaluate them in a Session/Graph context.
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

<h5>Après avoir examiné nos données ci-dessus, nous constatons que le texte brut contient des balises de rupture HTML de la forme '<br />'. Ces balises ne seront pas supprimées par le normalisateur par défaut (qui ne dépouille pas le HTML). Pour cette raison, nous devrons créer une fonction de normalisation personnalisée.<h5>
 <h5>Maintenant que nous avons notre normalisation personnalisée, nous pouvons instancier notre couche de vectorisation de texte. Nous utilisons cette couche pour normaliser, diviser et convertir les chaînes de caractères en nombres entiers, donc nous définissons notre 'output_mode' à 'int'.
nous utilisons la fonction de division par défaut, et la normalisation personnalisée définie ci-dessus.
Nous définissons également une longueur de séquence maximale explicite, puisque les CNNs plus tard dans notre modèle ne supporteront pas les séquences irrégulières.<h5>

In [48]:
from tensorflow.keras.layers import TextVectorization
import string
import re

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )

# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Now that the vocab layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)

In [49]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)


<h5>Nous choisissons un simple convnet 1D en commençant par Embedding layer<h5>

In [50]:
from tensorflow.keras import layers

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [51]:
epochs = 3

# Fit the model using the train and test datasets.
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3
625/625 [==============================] - 53s 83ms/step - loss: 0.5062 - accuracy: 0.7103 - val_loss: 0.3122 - val_accuracy: 0.8686
Epoch 2/3
625/625 [==============================] - 46s 74ms/step - loss: 0.2271 - accuracy: 0.9099 - val_loss: 0.3229 - val_accuracy: 0.8722
Epoch 3/3
625/625 [==============================] - 46s 74ms/step - loss: 0.1147 - accuracy: 0.9582 - val_loss: 0.4118 - val_accuracy: 0.8680


In [52]:
model.evaluate(test_ds)

782/782 [==============================] - 19s 24ms/step - loss: 0.4036 - accuracy: 0.8666


[0.40357086062431335, 0.866599977016449]

<h5>Si nous voulons obtenir un modèle capable de traiter des chaînes de caractères brutes, nous pouvons simplement créer un nouveau modèle (en utilisant les poids que nous venons d'entraîner) :<h5>

In [53]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

782/782 [==============================] - 11s 14ms/step - loss: 0.4036 - accuracy: 0.8666


[0.4035712778568268, 0.866599977016449]